In [19]:
import pandas as pd
import psycopg2

# Load cleaned CSV
df = pd.read_csv("../data/processed/clean_reviews.csv")

In [20]:
df.columns 
print(df.shape)
print(df.head())
df['date'] = pd.to_datetime(df['date'], errors='coerce')



(326, 14)
                                              review  rating        date  \
0                   maaliif daddafee install gaafata       3  2025-11-28   
1                                           good app       5  2025-11-28   
2  This application is very important and advanta...       5  2025-11-27   
3                          why didn't work this app?       1  2025-11-27   
4      The app makes our life easier. Thank you CBE!       5  2025-11-27   

                          bank             source  \
0  Commercial Bank of Ethiopia  Google Play Store   
1  Commercial Bank of Ethiopia  Google Play Store   
2  Commercial Bank of Ethiopia  Google Play Store   
3  Commercial Bank of Ethiopia  Google Play Store   
4  Commercial Bank of Ethiopia  Google Play Store   

                                        clean_review   slow  crash  login  \
0                   maaliif daddafee install gaafata  False  False  False   
1                                           good app  False 

In [21]:
# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="bank_reviews",
    user="postgres",
    password="Wakgari@123",  # Replace with your password
    host="localhost",
    port="5432"
)
cur = conn.cursor()

# --- Insert banks (avoid duplicates) ---
banks = df[['bank']].drop_duplicates()
for _, row in banks.iterrows():
    cur.execute("""
        INSERT INTO banks (bank_name, app_name)
        VALUES (%s, %s)
        ON CONFLICT (bank_name) DO NOTHING;
    """, (row['bank'], row['bank']))  # using bank name as app_name placeholder

conn.commit()

# --- Insert reviews ---
for _, row in df.iterrows():
    cur.execute("""
        INSERT INTO reviews (bank_id, review_text, rating, review_date, sentiment_label, sentiment_score, source)
        VALUES (
            (SELECT bank_id FROM banks WHERE bank_name = %s),
            %s, %s, %s, %s, %s, %s
        );
    """, (
        row['bank'],
        row['clean_review'],  # using clean_review for text
        row['rating'],
        row['date'],
        row['sentiment'],      # sentiment_label
        None,                  # sentiment_score (if not available yet)
        row['source']
    ))

conn.commit()

# --- Verify insertion ---
cur.execute("SELECT COUNT(*) FROM reviews;")
total_reviews = cur.fetchone()[0]
print(f"Total reviews inserted: {total_reviews}")

cur.close()
conn.close()


Total reviews inserted: 978
